In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Imports

In [2]:
# External libraries
import os as so
import sys as s
import pathlib as pl
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader, ConcatDataset
import torcheval
from torcheval.metrics import MulticlassF1Score, Mean
import optuna as opt
import torchvision as tn
import sklearn as sn
from sklearn.metrics import f1_score
import pandas as ps
import numpy as ny
import typing as t
import pathlib as pl
import matplotlib.pyplot as pt
import random as rng
from tqdm import tqdm
import tqdm as tm
from pprint import pprint
from git import Repo

In [3]:
# Add local package to path
if (p := pl.Path(so.getcwd(), '..').absolute().as_posix()) not in s.path:
    s.path.append(p)

# Local imports
from gic import *
from gic.tune import HyperParameterSampler, ClassificationTrainer
from gic.data import load_data, GenImageDataset
from gic.model import ResCNN

wandb: Currently logged in as: invokariman (castelvaar). Use `wandb login --relogin` to force relogin
/home/invokariman/Projects/git/ub-g21-deeplearning/projects/gic/notebooks/../gic/__init__.py:58: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wn_callback = WeightsAndBiasesCallback(
/home/invokariman/Projects/git/ub-g21-deeplearning/projects/gic/notebooks/../gic/tune.py:98: ExperimentalWarning: track_in_wandb is experimental (supported from v3.0.0). The interface can change in the future.
  @forward_self(wn_callback.track_in_wandb())


HyperParameter Optimization

In [4]:
# Search hyperparameters in a subspace
sampler = HyperParameterSampler(lambda trial: {
    'batch_size': trial.suggest_int('batch_size', 8, 32, step=16),
    'optimizer': trial.suggest_categorical('optimizer', ['Adam', 'AdamW']),
    'lr': trial.suggest_float('lr', 1e-4, 4e-3),
    'epochs': trial.suggest_int('epochs', 40, 60),
    'weight_decay': trial.suggest_float('weight_decay', 1e-6, 1e-3),
    'pool': trial.suggest_categorical('pool', ['max', 'avg']),
    'dropout1d': trial.suggest_float('dense_dropout', 0.2, 0.6),
    'dropout2d': trial.suggest_float('conv_dropout', 0.3, 0.6),
    'conv_chan': trial.suggest_int('conv_chan', 16, 32, step=8),
    'dens_chan': trial.suggest_int('dens_chan', 128, 1024, 512),
    'activ_fn': trial.suggest_categorical('activ', ['ReLU', 'SiLU', 'GELU'])
})

# Find the best model using a trainer
trainer = ClassificationTrainer(
    model=ResCNN,
    seed=SEED,
    hps=sampler,
    device=DEVICE,
    dataset_path=DATA_PATH,
    num_workers=num_workers,
    prefetch_factor=prefetch_factor,
)

In [5]:
# Search hyperparams
cutoff = opt.pruners.HyperbandPruner()
searcher = opt.samplers.TPESampler(n_startup_trials=10)
search = opt.create_study(direction='maximize', storage=db_uri, sampler=searcher, pruner=cutoff)
search.optimize(trainer, n_trials=30, callbacks=[wn_callback])
wn.finish()

[I 2023-12-01 01:44:12,753] A new study created in RDB with name: no-name-cbeda2d6-774a-4624-8fe2-bcecb1c00c87
wandb: WARNING Path ../tracking/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path ../tracking/wandb/ wasn't writable, using system temp directory


/home/invokariman/.cache/pypoetry/virtualenvs/gic-BfYgXNhZ-py3.11/lib/python3.11/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [8, 32] and step=16, but the range is not divisible by `step`. It will be replaced by [8, 24].
  warnings.warn(
/home/invokariman/.cache/pypoetry/virtualenvs/gic-BfYgXNhZ-py3.11/lib/python3.11/site-packages/optuna/distributions.py:700: UserWarning: The distribution is specified by [128, 1024] and step=512, but the range is not divisible by `step`. It will be replaced by [128, 640].
  warnings.warn(
epoch:  98%|█████████▊| 44/45 [05:49<00:07,  7.95s/it]
[W 2023-12-01 01:50:03,357] Trial 0 failed with parameters: {'batch_size': 8, 'optimizer': 'AdamW', 'lr': 0.00013042084716234596, 'epochs': 45, 'weight_decay': 0.00063217499863753, 'pool': 'avg', 'dense_dropout': 0.5908077569215953, 'conv_dropout': 0.43606675395538697, 'conv_chan': 32, 'dens_chan': 128, 'activ': 'SiLU'} because of the following error: KeyboardInterrupt()

KeyboardInterrupt: 

In [11]:
search.best_params

{'batch_size': 16,
 'optimizer': 'Adam',
 'lr': 0.00042881479787811055,
 'epochs': 27,
 'weight_decay': 0.0004010257351593616,
 'poo': 'avg',
 'dense_dropout': 0.0736934008960651,
 'conv_dropout': 0.29165493134158693,
 'conv_chan': 24,
 'dens_chan': 512,
 'activ': 'LeakyReLU'}

Final Training

In [7]:
# Use best model parameters
params = {
    'batch_size': 16,
    'optimizer': 'AdamW',
    'lr': 3e-4,
    'epochs': 50,
    'weight_decay': 5e-5,
    'pool': 'avg',
    'dense_dropout': 0.3,
    'conv_dropout': 0.4,
    'conv_chan': 32,
    'dens_chan': 512,
    'activ': 'SiLU'
}

# Train on `almost` the whole labeled dataset
# trainer.train(params)
# wn.finish()

Perform evaluation

In [8]:
preds: t.List[int] = trainer.eval(params).tolist()
preds[:10]

test_batch:   0%|          | 0/313 [00:01<?, ?it/s]


[67, 42, 40, 13, 69, 45, 24, 54, 26, 98]

In [9]:
len(preds)

5000

Save Submission

In [19]:
test_data = GenImageDataset(DATA_PATH, 'test', True)
dataf = {'Image': [], 'Class': []}
dataf['Class'] = preds
test_data._GenImageDataset__data['Class'] = preds
test_data._GenImageDataset__data.to_csv(SUBMISSION_PATH, index=False)